In [13]:
import plotly.plotly as py
import plotly.graph_objs as go
import plotly
import plotly.io as pio
import pandas as pd
import requests
import zipcodes
from pprint import pprint
from sklearn.preprocessing import scale
from sklearn import preprocessing
import os

min_max_scaler = preprocessing.MinMaxScaler()

In [15]:

def lat(row):
    zips = zipcodes.matching(row['Mail Zipcode'])
    for zip_code in zips:
        if zip_code['city'] == "SAN FRANCISCO":
            return str(zip_code['lat'])
        
def long(row):
    zips = zipcodes.matching(row['Mail Zipcode'])
    for zip_code in zips:
        if zip_code['city'] == "SAN FRANCISCO":
            return str(zip_code['long'])
        



df = pd.read_csv("/Users/nigel/SFTax/sf_new_business_neighborhood_price_appreciation-shifted.csv")
df.head()
df['Mail Zipcode'] = df['Mail Zipcode'].astype(str)
df['lat'] = df.apply(lat, axis=1)
df['long'] = df.apply(long, axis=1)
df['scaled new business count'] = scale(df['new business count'])
df['scaled pct change'] = scale(df['pct_change'])
df.head()
# df = df[(df['pct_change'] < 1) & (df['pct_change'] > -1)] 

,Unnamed: 0,ROLLYEAR,Mail Zipcode,new business count,pct_change,median land value price,lat,long,scaled new business count,scaled pct change
0,152,2014,94110,610.0,0.062268,242810.0,37.74,-122.41,4.805456,-0.034800
1,110,2014,94103,426.0,0.019782,308844.0,37.77,-122.41,3.093925,-0.325676
2,151,2013,94110,418.0,0.092849,228577.0,37.74,-122.41,3.019511,0.174571
3,131,2014,94107,412.0,0.031363,323023.0,37.76,-122.39,2.963700,-0.246385
4,150,2012,94110,399.0,0.104932,209157.0,37.74,-122.41,2.842777,0.257297


In [16]:
years = [2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017]

In [18]:
import plotly.plotly as py
import plotly.graph_objs as go

if not os.path.exists('mapcharts'):
    os.mkdir('mapcharts')


plotly.tools.set_credentials_file(username='graphdude', api_key='Cg2SFmNYy4XfTaGHTHro')

mapbox_access_token = 'pk.eyJ1IjoibmlnZWxoYWxsMSIsImEiOiJjanc2enhqZ3Axb3pvNDRwY3Rlam5rdmpjIn0.Mm898m2mXrYY0K-2JxStEg'

plots = []

# def plot_year(year):

year = 2013

df1 = df[df['ROLLYEAR'] == year]



for i, row in df1.iterrows():
#     print(row['lat'])
    plots.append(go.Scattermapbox(
        lat=[row['lat']],
        lon=[row['long']],
        mode='markers',
        marker=go.scattermapbox.Marker(
            color = 'blue',
            opacity=.5,
            size=(1+row['scaled new business count'])*10
        ),
        name=row['Mail Zipcode'],
        text=['San Francisco'],
    ))

for i, row in df1.iterrows():

    size = (1+row['pct_change'])**20
    color = 'green' if row['pct_change'] > 0 else "red"
#     print(row['lat'])
    plots.append(go.Scattermapbox(
        lat=[row['lat']],
        lon=[row['long']],
        mode='markers',
        marker=go.scattermapbox.Marker(
            color = color,
            opacity=.5,
            size=25 if size > 25 else size
        ),
        text=['San Francisco'],
    ))



layout = go.Layout(
    autosize=True,
    hovermode='closest',
    mapbox=go.layout.Mapbox(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=go.layout.mapbox.Center(
            lat=37.74,
            lon=-122.4241
        ),
        pitch=0,
        zoom=10.50
    ),
    
)

print(df1.sort_values("pct_change", ascending=False).head())
fig = go.Figure(data=plots, layout=layout)
py.iplot(fig, filename='New Business and Appreciation {} Mapbox'.format(year))
# pio.write_image(fig, 'mapcharts/{}.png'.format(year))

     Unnamed: 0  ROLLYEAR Mail Zipcode  new business count  pct_change  \
307         355      2013        95070                 3.0    0.302956   
173         290      2013        94403                29.0    0.288327   
246         299      2013        94549                 8.0    0.283657   
189         283      2013        94402                18.0    0.280797   
52          158      2013        94111               204.0    0.269655   

     median land value price    lat     long  scaled new business count  \
307                 245331.0   None     None                  -0.840735   
173                  91935.0   None     None                  -0.598888   
246                 179495.0   None     None                  -0.794226   
189                 176768.5   None     None                  -0.701208   
52                  339573.0  37.79  -122.39                   1.028926   

     scaled pct change  
307           1.613051  
173           1.512893  
246           1.480920  
189 